In [1]:
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [2]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

In [3]:
###### 读取数据 
stock_data = pd.read_csv('stock_data_normal_u.csv',header=None)
stock_lable = pd.read_csv('stock_data_lable_normal.txt',header=None)

In [4]:
#print(stock_lable.head(10))

In [6]:
lab_count = stock_lable.shape[0]
x = stock_data.loc[0:59999].values
print(x.shape)
x = x.reshape(-1,66)
print(x.shape)
y = stock_lable.loc[0:9999,2].values
y = y.reshape(-1,1)
print(y.shape)
#x = x / 2
x = np.insert(x,0,1,axis=1)

(60000, 11)
(10000, 66)
(10000, 1)


In [7]:
def J_softmax(h,y):
    a = 1e-10
    b = y*np.log(h+a)
    return b.sum() / 60000

In [8]:
def J_sigmond(h,y):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / 10000

In [9]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1.txt",delimiter=',')
w2 = np.loadtxt("init_w2.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.8
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[-0.80246953  0.18560175 -0.00419939 -0.35403614  0.85984978]
 [-0.38040226 -0.37840635  0.09163621  0.12378902 -0.90720175]
 [-0.73360228  0.53896761 -0.03028989 -0.92204824  1.00481472]
 [ 0.18550521  0.39085074 -0.35709077 -0.34868764  0.85515554]
 [-0.12533647 -0.84822099  0.40565257  0.90414357  0.91487373]
 [ 0.51257121 -0.24554629 -0.06185238  0.4880797  -0.62664755]
 [ 0.39126569 -1.07643386  0.6079962  -0.04509457 -0.12263014]
 [ 0.71674896  0.57421826 -0.62683209 -1.15799362  0.41696275]
 [-0.36726845  0.88314125  0.95786877  0.25830563  0.43386327]
 [-0.08686252 -1.12690207  0.07127851 -0.37811258  0.35322541]]
[[-1.54260951]
 [ 0.94379049]
 [ 0.97680887]
 [ 0.21875728]
 [-0.73366289]
 [-0.74681891]
 [ 0.86164704]
 [ 1.24734318]
 [ 0.80785735]
 [ 0.84649108]]


In [106]:
"""
构建一个三层神经网络
一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段
取一般数据为训练样本数

数据归一化，分字段进行，然后 reshape(-1,1)
隐藏层 200 个神经元,输入到隐藏层，用reLu
输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy
w1 的 大小为 67*200
w2 的 大小为 201*1

"""

'\n构建一个三层神经网络\n一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段\n取一般数据为训练样本数\n\n数据归一化，分字段进行，然后 reshape(-1,1)\n隐藏层 200 个神经元,输入到隐藏层，用reLu\n输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy\nw1 的 大小为 67*200\nw2 的 大小为 201*1\n\n'

In [32]:
for i in range(700):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(x,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 10000
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 10000
    print('----------cost is -------------',J_sigmond(hpfs_output,y))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')

print('----------------done--------------------------')  

----------cost is ------------- 0.027610314000837535
----------cost is ------------- 0.027599319555043116
----------cost is ------------- 0.02758833268210878
----------cost is ------------- 0.02757735337480337
----------cost is ------------- 0.027566381625904406
----------cost is ------------- 0.027555417428198132
----------cost is ------------- 0.0275444607744795
----------cost is ------------- 0.027533511657552096
----------cost is ------------- 0.0275225700702282
----------cost is ------------- 0.027511636005328736
----------cost is ------------- 0.02750070945568327
----------cost is ------------- 0.02748979041412998
----------cost is ------------- 0.027478878873515696
----------cost is ------------- 0.02746797482669582
----------cost is ------------- 0.027457078266534346
----------cost is ------------- 0.02744618918590386
----------cost is ------------- 0.02743530757768548
----------cost is ------------- 0.02742443343476896
----------cost is ------------- 0.027413566750052475
-----

----------cost is ------------- 0.0259725513128082
----------cost is ------------- 0.025962662475583854
----------cost is ------------- 0.02595278017517132
----------cost is ------------- 0.02594290440556597
----------cost is ------------- 0.025933035160770133
----------cost is ------------- 0.025923172434793137
----------cost is ------------- 0.025913316221651227
----------cost is ------------- 0.025903466515367603
----------cost is ------------- 0.025893623309972413
----------cost is ------------- 0.02588378659950271
----------cost is ------------- 0.025873956378002463
----------cost is ------------- 0.02586413263952259
----------cost is ------------- 0.025854315378120822
----------cost is ------------- 0.025844504587861875
----------cost is ------------- 0.025834700262817263
----------cost is ------------- 0.025824902397065395
----------cost is ------------- 0.025815110984691546
----------cost is ------------- 0.025805326019787845
----------cost is ------------- 0.025795547496453214

----------cost is ------------- 0.024478559245900512
----------cost is ------------- 0.024469638684248528
----------cost is ------------- 0.02446072378625432
----------cost is ------------- 0.024451814546909537
----------cost is ------------- 0.02444291096121144
----------cost is ------------- 0.02443401302416287
----------cost is ------------- 0.024425120730772278
----------cost is ------------- 0.024416234076053714
----------cost is ------------- 0.024407353055026768
----------cost is ------------- 0.024398477662716606
----------cost is ------------- 0.024389607894153995
----------cost is ------------- 0.024380743744375228
----------cost is ------------- 0.02437188520842213
----------cost is ------------- 0.02436303228134209
----------cost is ------------- 0.024354184958188037
----------cost is ------------- 0.024345343234018414
----------cost is ------------- 0.024336507103897155
----------cost is ------------- 0.02432767656289375
----------cost is ------------- 0.02431885160608316


----------cost is ------------- 0.0231524448394902
----------cost is ------------- 0.023144350566525796
----------cost is ------------- 0.023136261239291436
----------cost is ------------- 0.023128176853569174
----------cost is ------------- 0.02312009740514555
----------cost is ------------- 0.0231120228898117
----------cost is ------------- 0.02310395330336328
----------cost is ------------- 0.023095888641600535
----------cost is ------------- 0.023087828900328216
----------cost is ------------- 0.02307977407535561
----------cost is ------------- 0.02307172416249652
----------cost is ------------- 0.023063679157569267
----------cost is ------------- 0.023055639056396734
----------cost is ------------- 0.02304760385480624
----------cost is ------------- 0.023039573548629642
----------cost is ------------- 0.02303154813370331
----------cost is ------------- 0.023023527605868063
----------cost is ------------- 0.023015511960969234
----------cost is ------------- 0.02300750119485663
----

----------cost is ------------- 0.021939647465862553
----------cost is ------------- 0.02193228076647064
----------cost is ------------- 0.021924918402887725
----------cost is ------------- 0.021917560371544496
----------cost is ------------- 0.021910206668875373
----------cost is ------------- 0.021902857291318516
----------cost is ------------- 0.0218955122353158
----------cost is ------------- 0.02188817149731279
----------cost is ------------- 0.021880835073758795
----------cost is ------------- 0.021873502961106843
----------cost is ------------- 0.021866175155813607
----------cost is ------------- 0.021858851654339508
----------cost is ------------- 0.021851532453148637
----------cost is ------------- 0.021844217548708786
----------cost is ------------- 0.021836906937491418
----------cost is ------------- 0.021829600615971684
----------cost is ------------- 0.0218222985806284
----------cost is ------------- 0.02181500082794406
----------cost is ------------- 0.021807707354404814


In [14]:
### 先用训练集验证

index = random.randint(0,999)
x_train_test = x[index]
x_train_test = x_train_test.reshape(1,-1)
#x_train_test = np.insert(x_train_test,0,1,axis=0)
#index = 300
y_test = y[index]
print('--序号--',index)
print('----输入-----',x_train_test.shape)
#print(x_train_test)
hid_input = np.matmul(x_train_test,w1)
hid_output = np.tanh(hid_input)
hid_output = np.insert(hid_output,0,1,axis=1)
hpfs_input = np.matmul(hid_output,w2)
hpfs_output = sigmond(hpfs_input)
print('-----输出-----',y_test,hpfs_output)

--序号-- 176
[[ 1.      -1.45747  0.91287  1.33631 -1.68882  1.74758  1.69205 -1.35561
  -0.7398   1.64768 -0.40825  0.87775  0.51717 -0.91287 -1.33631  0.18765
  -1.16331 -0.38899 -0.89083 -0.7398   0.63889 -0.40825 -0.62696 -0.6112
  -0.91287 -0.26726 -0.18765 -0.35168 -0.83808 -0.25176 -0.6576   0.03363
  -0.40825 -0.62696 -0.32911 -0.91287 -0.80178 -0.18765 -0.35168 -1.03678
   0.44542 -0.2466  -0.57164 -0.40825 -0.62696  0.51717  0.91287  0.26726
   0.56294  0.47079  0.1361   1.02639  0.6576  -0.7734  -0.40825 -0.62696
   1.36344  0.91287  0.80178  1.31353 -0.35168  0.4357   1.02639  1.72619
  -0.97515  2.04124  1.6301 ]]
-----输出----- [0] [[0.03359639]]


In [22]:
#index = random.randint(1501,2100)
error_data_list  = []
all_list = []
error_count = 0
compary_np = np.array(np.zeros(20000)).reshape(2,10000)
#print(stock_lable.info())
#index = 300 
for index in range(10000,20000): 
    x_test = stock_data.loc[index*6:index*6+5].values 
    x_test = x_test.reshape(1,-1) 
    x_test = np.insert(x_test,0,1,axis=1) 
    #print('--序号--',index)

    #print('----输入-----',x_test)
    y_test = stock_lable.loc[index,2]
    hid_input = np.matmul(x_test,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=1)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    output = hpfs_output[0][0]
    y_value = y_test
    #print(output)
    if(output > 0.5 and y_value ==1) or (output<0.5 and y_value ==0):
        print('-----输出-----',y_test,hpfs_output)
    else:
        print('-----输出-----',y_test,hpfs_output)
        #print(stock_lable.loc[index])
        error_data_list.append(str(stock_lable.loc[index,0])+','+str(stock_lable.loc[index,1])+','+str(stock_lable.loc[index,2])+','+str(output)+'\\n')
        #error_data.append(stock_lable.loc[index],ignore_index='True')
        error_count = error_count + 1
    #print('-----输出-----',y_test,hpfs_output)
        compary_np[0:2,index-10000]=np.array([y_test,hpfs_output])
    all_list.append(str(stock_lable.loc[index,0])+','+str(stock_lable.loc[index,1])+','+str(stock_lable.loc[index,2])+','+str(output)+'\\n')
np.savetxt('compary.txt',compary_np,delimiter=',') 
    
print('-----------done-----------------')

-----输出----- 0 [[0.00477366]]
-----输出----- 1 [[0.98933631]]
-----输出----- 0 [[0.01592353]]
-----输出----- 0 [[0.00243467]]
-----输出----- 0 [[2.88255187e-05]]
-----输出----- 0 [[4.58360234e-06]]
-----输出----- 0 [[0.00478195]]
-----输出----- 1 [[0.99655284]]
-----输出----- 0 [[0.00022381]]
-----输出----- 0 [[4.7379199e-07]]
-----输出----- 1 [[0.99274307]]
-----输出----- 0 [[0.99999882]]
-----输出----- 0 [[0.00110199]]
-----输出----- 0 [[0.00436761]]
-----输出----- 1 [[0.99831065]]
-----输出----- 0 [[0.00043826]]
-----输出----- 0 [[0.4935094]]
-----输出----- 0 [[0.24496486]]
-----输出----- 0 [[0.31343808]]
-----输出----- 0 [[0.45003949]]
-----输出----- 1 [[0.99998918]]
-----输出----- 0 [[0.32617764]]
-----输出----- 0 [[8.6838126e-06]]
-----输出----- 1 [[0.7347513]]
-----输出----- 0 [[1.56407726e-06]]
-----输出----- 0 [[0.9930112]]
-----输出----- 0 [[0.00011234]]
-----输出----- 1 [[0.81334054]]
-----输出----- 0 [[0.0069743]]
-----输出----- 0 [[0.00084426]]
-----输出----- 0 [[0.00062224]]
-----输出----- 1 [[0.00195736]]
-----输出----- 0 [[0.0748711

-----输出----- 1 [[0.00719056]]
-----输出----- 0 [[0.04645646]]
-----输出----- 0 [[3.09015113e-05]]
-----输出----- 0 [[0.82066269]]
-----输出----- 0 [[0.00241148]]
-----输出----- 0 [[5.41101253e-11]]
-----输出----- 0 [[0.00033702]]
-----输出----- 0 [[5.88741162e-06]]
-----输出----- 0 [[0.0052707]]
-----输出----- 1 [[0.99690567]]
-----输出----- 0 [[0.00072267]]
-----输出----- 0 [[0.18737891]]
-----输出----- 0 [[0.00256313]]
-----输出----- 0 [[0.00167303]]
-----输出----- 1 [[0.77159796]]
-----输出----- 1 [[0.0748912]]
-----输出----- 1 [[0.000323]]
-----输出----- 0 [[0.00392926]]
-----输出----- 1 [[0.21420941]]
-----输出----- 1 [[0.84292147]]
-----输出----- 0 [[2.4022735e-06]]
-----输出----- 0 [[0.00014369]]
-----输出----- 0 [[0.30926399]]
-----输出----- 0 [[0.28373747]]
-----输出----- 0 [[1.20788927e-05]]
-----输出----- 0 [[0.00041861]]
-----输出----- 0 [[0.04159038]]
-----输出----- 0 [[0.05229593]]
-----输出----- 1 [[0.9943797]]
-----输出----- 0 [[0.00958977]]
-----输出----- 0 [[0.58479047]]
-----输出----- 0 [[0.6310812]]
-----输出----- 0 [[0.10128987

-----输出----- 1 [[0.48601488]]
-----输出----- 0 [[0.02421572]]
-----输出----- 0 [[0.01952133]]
-----输出----- 0 [[0.00075068]]
-----输出----- 0 [[0.05039401]]
-----输出----- 0 [[0.01039106]]
-----输出----- 1 [[0.00555258]]
-----输出----- 0 [[0.00149851]]
-----输出----- 0 [[0.00333293]]
-----输出----- 0 [[0.30950001]]
-----输出----- 0 [[0.00085468]]
-----输出----- 0 [[0.28285917]]
-----输出----- 1 [[0.78718225]]
-----输出----- 0 [[1.21916493e-06]]
-----输出----- 1 [[0.82426574]]
-----输出----- 0 [[0.00463552]]
-----输出----- 1 [[0.99628665]]
-----输出----- 1 [[0.99993816]]
-----输出----- 0 [[0.00210278]]
-----输出----- 0 [[3.18947502e-05]]
-----输出----- 0 [[0.00505242]]
-----输出----- 0 [[0.02783469]]
-----输出----- 1 [[4.21426724e-05]]
-----输出----- 0 [[0.00101994]]
-----输出----- 0 [[0.00108398]]
-----输出----- 0 [[4.45699068e-06]]
-----输出----- 1 [[0.91322154]]
-----输出----- 0 [[0.01892494]]
-----输出----- 1 [[0.99988802]]
-----输出----- 0 [[0.00010684]]
-----输出----- 1 [[0.97845958]]
-----输出----- 0 [[0.1032878]]
-----输出----- 1 [[0.999950

-----输出----- 0 [[0.04410182]]
-----输出----- 0 [[6.51086576e-05]]
-----输出----- 1 [[6.7081768e-06]]
-----输出----- 0 [[0.0003973]]
-----输出----- 1 [[0.93894473]]
-----输出----- 0 [[5.66506642e-05]]
-----输出----- 0 [[4.78368156e-05]]
-----输出----- 0 [[0.98795775]]
-----输出----- 0 [[0.00552063]]
-----输出----- 0 [[0.87183278]]
-----输出----- 1 [[0.01510366]]
-----输出----- 0 [[0.73621687]]
-----输出----- 0 [[0.00226514]]
-----输出----- 0 [[0.00901308]]
-----输出----- 1 [[0.02594398]]
-----输出----- 0 [[0.08063971]]
-----输出----- 0 [[0.99994292]]
-----输出----- 0 [[2.69014422e-05]]
-----输出----- 1 [[0.9008083]]
-----输出----- 0 [[0.04387222]]
-----输出----- 0 [[0.04384495]]
-----输出----- 1 [[0.72548212]]
-----输出----- 1 [[0.1549192]]
-----输出----- 1 [[7.73022304e-05]]
-----输出----- 0 [[0.94444879]]
-----输出----- 0 [[0.99929887]]
-----输出----- 1 [[0.55917729]]
-----输出----- 0 [[0.00859504]]
-----输出----- 1 [[0.94057889]]
-----输出----- 0 [[0.98158084]]
-----输出----- 0 [[0.98287519]]
-----输出----- 1 [[0.00608093]]
-----输出----- 0 [[0.0

-----输出----- 1 [[0.00010673]]
-----输出----- 0 [[0.06435091]]
-----输出----- 0 [[2.76512017e-06]]
-----输出----- 0 [[0.00014565]]
-----输出----- 0 [[0.00372665]]
-----输出----- 1 [[0.82568334]]
-----输出----- 0 [[0.00607511]]
-----输出----- 0 [[0.03403884]]
-----输出----- 0 [[3.65182974e-07]]
-----输出----- 0 [[0.55945864]]
-----输出----- 0 [[0.80593361]]
-----输出----- 1 [[0.83710192]]
-----输出----- 0 [[0.00052717]]
-----输出----- 0 [[0.00023123]]
-----输出----- 0 [[0.0290794]]
-----输出----- 0 [[0.03197929]]
-----输出----- 1 [[0.40379257]]
-----输出----- 0 [[0.02046864]]
-----输出----- 0 [[0.04701402]]
-----输出----- 0 [[0.00225647]]
-----输出----- 1 [[0.55496575]]
-----输出----- 0 [[1.61149388e-05]]
-----输出----- 0 [[0.48333255]]
-----输出----- 1 [[0.99998898]]
-----输出----- 0 [[0.01073432]]
-----输出----- 0 [[0.00082273]]
-----输出----- 0 [[8.38011346e-05]]
-----输出----- 0 [[0.9288815]]
-----输出----- 0 [[0.00088896]]
-----输出----- 1 [[0.26300832]]
-----输出----- 0 [[0.01876461]]
-----输出----- 0 [[0.99908364]]
-----输出----- 0 [[4.7309473

...

-----输出----- 0 [[0.32194039]]
-----输出----- 0 [[0.00028772]]
-----输出----- 0 [[0.0006295]]
-----输出----- 0 [[0.30799143]]
-----输出----- 1 [[0.99782057]]
-----输出----- 0 [[0.00049497]]
-----输出----- 0 [[0.00075506]]
-----输出----- 0 [[0.00090905]]
-----输出----- 0 [[0.02133275]]
-----输出----- 1 [[0.89477165]]
-----输出----- 0 [[5.58511153e-05]]
-----输出----- 0 [[0.00015441]]
-----输出----- 0 [[0.82996071]]
-----输出----- 0 [[0.00015241]]
-----输出----- 0 [[3.8943791e-05]]
-----输出----- 0 [[0.03378277]]
-----输出----- 0 [[0.0030544]]
-----输出----- 0 [[0.00520983]]
-----输出----- 0 [[0.01085524]]
-----输出----- 0 [[0.0010982]]
-----输出----- 1 [[0.00897019]]
-----输出----- 0 [[3.76317183e-06]]
-----输出----- 0 [[0.00098141]]
-----输出----- 0 [[0.00963204]]
-----输出----- 1 [[0.99672638]]
-----输出----- 0 [[0.00118581]]
-----输出----- 1 [[0.61829088]]
-----输出----- 1 [[0.08530194]]
-----输出----- 1 [[0.00340956]]
-----输出----- 0 [[0.74284416]]
-----输出----- 0 [[0.01209213]]
-----输出----- 0 [[0.01923504]]
-----输出----- 1 [[0.9529026]]
---

-----输出----- 0 [[0.10801986]]
-----输出----- 0 [[0.99629589]]
-----输出----- 0 [[0.77928693]]
-----输出----- 0 [[4.76169287e-05]]
-----输出----- 0 [[0.00233217]]
-----输出----- 0 [[0.00119261]]
-----输出----- 0 [[0.00209638]]
-----输出----- 0 [[0.66117796]]
-----输出----- 1 [[0.99998593]]
-----输出----- 1 [[0.71939258]]
-----输出----- 0 [[0.00177969]]
-----输出----- 0 [[0.00252756]]
-----输出----- 0 [[3.43859688e-05]]
-----输出----- 1 [[0.13936206]]
-----输出----- 0 [[5.50079934e-05]]
-----输出----- 0 [[0.14067751]]
-----输出----- 0 [[0.00355118]]
-----输出----- 1 [[0.99999806]]
-----输出----- 0 [[0.00358476]]
-----输出----- 0 [[0.98812909]]
-----输出----- 0 [[0.03544948]]
-----输出----- 0 [[0.00050029]]
-----输出----- 0 [[0.00168981]]
-----输出----- 1 [[0.99753438]]
-----输出----- 1 [[0.00589357]]
-----输出----- 0 [[0.03311171]]
-----输出----- 0 [[0.00025363]]
-----输出----- 0 [[1.20921014e-05]]
-----输出----- 0 [[0.01151199]]
-----输出----- 0 [[0.95360589]]
-----输出----- 1 [[0.89220538]]
-----输出----- 1 [[0.06406756]]
-----输出----- 0 [[4.23063

-----输出----- 0 [[0.92662708]]
-----输出----- 0 [[0.03502546]]
-----输出----- 1 [[0.99945447]]
-----输出----- 0 [[0.00013494]]
-----输出----- 0 [[1.67755915e-05]]
-----输出----- 0 [[0.00263704]]
-----输出----- 0 [[0.00022545]]
-----输出----- 0 [[0.01357471]]
-----输出----- 1 [[0.14051666]]
-----输出----- 1 [[0.05910857]]
-----输出----- 1 [[0.10598032]]
-----输出----- 0 [[0.02014152]]
-----输出----- 0 [[0.00162956]]
-----输出----- 0 [[3.71269084e-06]]
-----输出----- 0 [[0.00652411]]
-----输出----- 1 [[0.96011545]]
-----输出----- 0 [[0.0060016]]
-----输出----- 0 [[2.20469045e-07]]
-----输出----- 0 [[6.84621728e-07]]
-----输出----- 0 [[0.01097633]]
-----输出----- 0 [[0.00059441]]
-----输出----- 0 [[8.26761948e-06]]
-----输出----- 1 [[0.999892]]
-----输出----- 0 [[8.98607582e-07]]
-----输出----- 0 [[0.02249903]]
-----输出----- 0 [[0.00335369]]
-----输出----- 0 [[0.67013726]]
-----输出----- 1 [[0.9573604]]
-----输出----- 0 [[0.00665495]]
-----输出----- 0 [[0.0315447]]
-----输出----- 1 [[0.89265873]]
-----输出----- 0 [[0.86538511]]
-----输出----- 0 [[2.63

-----输出----- 0 [[0.01085179]]
-----输出----- 0 [[0.002163]]
-----输出----- 0 [[0.01877474]]
-----输出----- 0 [[0.90942789]]
-----输出----- 0 [[0.75825943]]
-----输出----- 0 [[0.00024514]]
-----输出----- 1 [[0.99999077]]
-----输出----- 0 [[0.00078219]]
-----输出----- 0 [[9.53244219e-05]]
-----输出----- 0 [[0.16562208]]
-----输出----- 0 [[1.66527709e-06]]
-----输出----- 0 [[0.40144725]]
-----输出----- 1 [[0.96287512]]
-----输出----- 0 [[6.7771108e-05]]
-----输出----- 0 [[0.00115033]]
-----输出----- 0 [[0.01035087]]
-----输出----- 0 [[0.02216795]]
-----输出----- 0 [[8.81506618e-05]]
-----输出----- 0 [[0.9642274]]
-----输出----- 0 [[0.05515127]]
-----输出----- 0 [[4.40356595e-06]]
-----输出----- 0 [[0.980839]]
-----输出----- 0 [[0.01240489]]
-----输出----- 0 [[0.12733044]]
-----输出----- 0 [[0.00153047]]
-----输出----- 0 [[3.84391499e-05]]
-----输出----- 0 [[0.02759727]]
-----输出----- 0 [[0.00212023]]
-----输出----- 0 [[0.00018361]]
-----输出----- 0 [[0.00662861]]
-----输出----- 0 [[0.01696864]]
-----输出----- 0 [[3.99069754e-06]]
-----输出----- 1 [[0

-----输出----- 1 [[0.00321592]]
-----输出----- 0 [[1.74986574e-08]]
-----输出----- 1 [[0.45306444]]
-----输出----- 0 [[2.82277707e-07]]
-----输出----- 1 [[0.04176781]]
-----输出----- 0 [[0.05555159]]
-----输出----- 0 [[3.45729713e-05]]
-----输出----- 1 [[0.99767971]]
-----输出----- 0 [[9.5742163e-06]]
-----输出----- 0 [[0.99754031]]
-----输出----- 0 [[0.00013877]]
-----输出----- 0 [[0.07441911]]
-----输出----- 0 [[0.0284595]]
-----输出----- 0 [[0.0002016]]
-----输出----- 0 [[8.03909492e-06]]
-----输出----- 0 [[0.50133529]]
-----输出----- 1 [[0.00028331]]
-----输出----- 0 [[0.02020083]]
-----输出----- 0 [[0.0657959]]
-----输出----- 0 [[0.00027598]]
-----输出----- 1 [[0.99999734]]
-----输出----- 0 [[0.00236533]]
-----输出----- 1 [[1.02581534e-05]]
-----输出----- 0 [[0.15401441]]
-----输出----- 0 [[1.49629142e-05]]
-----输出----- 0 [[0.23132123]]
-----输出----- 0 [[0.35858029]]
-----输出----- 0 [[0.01536986]]
-----输出----- 0 [[0.00289186]]
-----输出----- 0 [[0.00443988]]
-----输出----- 0 [[0.8762213]]
-----输出----- 0 [[0.02349796]]
-----输出----- 0 [[

In [25]:
print(error_data_list)

['300229,20191205,0,0.9999988193289777\\n', '300231,20191206,0,0.9930111964543283\\n', '300233,20190225,1,0.001957361313506635\\n', '300233,20190610,1,0.030577134455525155\\n', '300234,20190315,1,0.09697706066975845\\n', '300235,20191021,0,0.9411374116165802\\n', '300237,20190816,0,0.8102357147410056\\n', '300237,20191213,0,0.8307860047175953\\n', '300238,20190419,1,0.0014869785100809075\\n', '300239,20190401,0,0.8292092671832794\\n', '300241,20190711,0,0.7621511596642137\\n', '300241,20190723,1,0.14227869514547845\\n', '300244,20190620,1,0.02131026032134085\\n', '300244,20190918,0,0.6450057967309621\\n', '300245,20190815,0,0.9754372156809128\\n', '300249,20190218,1,0.000509417212768817\\n', '300249,20190816,1,0.09781865960587266\\n', '300251,20190225,0,0.8827494763834486\\n', '300251,20190618,1,5.56746914989243e-05\\n', '300251,20191025,1,0.10341795600464715\\n', '300252,20190902,0,0.9997524451460285\\n', '300253,20190814,1,0.06507032398821423\\n', '300253,20191011,0,0.897498238297509

In [24]:
print(all_list)

['300228,20190107,0,0.004773657148309154\\n', '300228,20190211,1,0.9893363088247181\\n', '300228,20190401,0,0.01592352625613565\\n', '300228,20190619,0,0.0024346746353752294\\n', '300228,20190827,0,2.8825518719715536e-05\\n', '300228,20191129,0,4.58360234385786e-06\\n', '300229,20190104,0,0.0047819493284825595\\n', '300229,20190211,1,0.9965528386005156\\n', '300229,20190412,0,0.00022381320272569702\\n', '300229,20190619,0,4.737919902644438e-07\\n', '300229,20190815,1,0.9927430729317983\\n', '300229,20191205,0,0.9999988193289777\\n', '300230,20190107,0,0.0011019911984890022\\n', '300230,20190121,0,0.004367610429787584\\n', '300230,20190201,1,0.9983106496477981\\n', '300230,20190401,0,0.00043826008399458814\\n', '300230,20190620,0,0.4935093996495988\\n', '300230,20190816,0,0.24496486478547566\\n', '300230,20191023,0,0.31343807902091825\\n', '300231,20190107,0,0.4500394921641205\\n', '300231,20190211,1,0.9999891834773076\\n', '300231,20190617,0,0.32617763558984064\\n', '300231,20190724,0,